In [ ]:
# Load environment variables and initialize Anthropic client for text editing tasks
# This cell sets up the SDK and loads API credentials from .env file

from dotenv import load_dotenv
from anthropic import Anthropic

# Load environment variables (including ANTHROPIC_API_KEY) from .env file
load_dotenv()

# Initialize the Anthropic client with credentials from environment
client = Anthropic()

# Specify the model to use for API requests - Sonnet 4.5 supports tool use
model = "claude-sonnet-4-5"


In [ ]:
# Helper functions for managing conversations and API interactions
# These utilities simplify message handling and API calls with tool support

from anthropic.types import Message


def add_user_message(messages, message):
    """
    Add a user message to the conversation history.
    
    Parameters:
    - messages: List of message dictionaries representing conversation history
    - message: Either a string, list of content blocks, or Anthropic Message object
    
    Returns: None (modifies messages list in place)
    
    Note: Handles both string and Message object inputs for flexibility
    """
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    """
    Add an assistant message to the conversation history.
    
    Parameters:
    - messages: List of message dictionaries representing conversation history
    - message: Either a string or Anthropic Message object containing assistant response
    
    Returns: None (modifies messages list in place)
    
    Note: Converts Anthropic Message objects to content blocks for consistency
    """
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    """
    Send a message to Claude and receive a response, with optional tool support.
    
    Parameters:
    - messages: List of message dictionaries representing conversation history
    - system: Optional system prompt to guide model behavior
    - temperature: Controls randomness of response (1.0 = default, lower = more deterministic)
    - stop_sequences: Optional list of strings to stop generation at
    - tools: Optional list of tool definitions to enable tool use
    
    Returns: Anthropic Message object containing the model's response
    
    The function builds parameters dynamically, only including tools and system 
    prompt if provided. Tool use enables the model to request file operations.
    """
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    # Add tools parameter if tools are provided
    if tools:
        params["tools"] = tools

    # Add system prompt if provided
    if system:
        params["system"] = system

    # Call the Anthropic API with the constructed parameters
    message = client.messages.create(**params)
    return message


def text_from_message(message):
    """
    Extract plain text content from an Anthropic Message object.
    
    Parameters:
    - message: Anthropic Message object with content blocks
    
    Returns: String containing all text blocks joined by newlines
    
    Note: Filters out non-text blocks (e.g., tool_use blocks) and returns only text content
    """
    return "\n".join([block.text for block in message.content if block.type == "text"])


In [ ]:
# Implementation of the TextEditorTool class for safe file operations
# Provides secure file viewing, creation, editing, and backup management

import os
import shutil
from typing import Optional, List


class TextEditorTool:
    """
    A secure text editor tool that allows safe file operations with automatic backups.
    
    Features:
    - Path validation to prevent directory traversal attacks
    - Automatic backup creation before file modifications
    - Support for viewing files with optional line range selection
    - String replacement with validation (prevents ambiguous replacements)
    - File creation with error handling
    - Line insertion with proper line numbering
    - Undo capability through backup restoration
    """
    
    def __init__(self, base_dir: str = "", backup_dir: str = ""):
        """
        Initialize the TextEditorTool with optional base and backup directories.
        
        Parameters:
        - base_dir: Root directory for file operations (defaults to current working directory)
        - backup_dir: Directory to store backups (defaults to .backups in base_dir)
        """
        self.base_dir = base_dir or os.getcwd()
        self.backup_dir = backup_dir or os.path.join(self.base_dir, ".backups")
        # Create backup directory if it doesn't exist
        os.makedirs(self.backup_dir, exist_ok=True)

    def _validate_path(self, file_path: str) -> str:
        """
        Validate and normalize file path to prevent directory traversal attacks.
        
        Parameters:
        - file_path: Relative path to validate
        
        Returns: Absolute normalized path
        
        Raises: ValueError if path is outside base_dir
        """
        abs_path = os.path.normpath(os.path.join(self.base_dir, file_path))
        if not abs_path.startswith(self.base_dir):
            raise ValueError(
                f"Access denied: Path '{file_path}' is outside the allowed directory"
            )
        return abs_path

    def _backup_file(self, file_path: str) -> str:
        """
        Create a timestamped backup of the file before modification.
        
        Parameters:
        - file_path: Absolute path to the file to backup
        
        Returns: Path to the created backup file, or empty string if file doesn't exist
        """
        if not os.path.exists(file_path):
            return ""
        # Create backup with timestamp to allow multiple versions
        file_name = os.path.basename(file_path)
        backup_path = os.path.join(
            self.backup_dir, f"{file_name}.{os.path.getmtime(file_path):.0f}"
        )
        shutil.copy2(file_path, backup_path)
        return backup_path

    def _restore_backup(self, file_path: str) -> str:
        """
        Restore a file from its most recent backup.
        
        Parameters:
        - file_path: Absolute path to the file to restore
        
        Returns: Success message
        
        Raises: FileNotFoundError if no backups exist
        """
        file_name = os.path.basename(file_path)
        # Find all backups for this file
        backups = [
            f for f in os.listdir(self.backup_dir) if f.startswith(file_name + ".")
        ]
        if not backups:
            raise FileNotFoundError(f"No backups found for {file_path}")

        # Restore the most recent backup (sorted by timestamp)
        latest_backup = sorted(backups, reverse=True)[0]
        backup_path = os.path.join(self.backup_dir, latest_backup)

        shutil.copy2(backup_path, file_path)
        return f"Successfully restored {file_path} from backup"

    def _count_matches(self, content: str, old_str: str) -> int:
        """
        Count occurrences of a string in content.
        
        Parameters:
        - content: Text content to search in
        - old_str: String to count occurrences of
        
        Returns: Number of matches found
        """
        return content.count(old_str)

    def view(self, file_path: str, view_range: Optional[List[int]] = None) -> str:
        """
        View file contents with optional line range selection.
        
        Parameters:
        - file_path: Relative path to file or directory
        - view_range: Optional [start_line, end_line] to view specific lines (-1 for end means last line)
        
        Returns: File contents with line numbers, or directory listing for directories
        
        Raises: FileNotFoundError, PermissionError, UnicodeDecodeError for various error conditions
        """
        try:
            abs_path = self._validate_path(file_path)

            # If path is a directory, list its contents
            if os.path.isdir(abs_path):
                try:
                    return "\n".join(os.listdir(abs_path))
                except PermissionError:
                    raise PermissionError(
                        "Permission denied. Cannot list directory contents."
                    )

            # Check if file exists
            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            # Read file content
            with open(abs_path, "r", encoding="utf-8") as f:
                content = f.read()

            # If view_range is specified, return only those lines
            if view_range:
                start, end = view_range
                lines = content.split("\n")

                # Handle -1 as "end of file"
                if end == -1:
                    end = len(lines)

                selected_lines = lines[start - 1 : end]

                # Format with line numbers
                result = []
                for i, line in enumerate(selected_lines, start):
                    result.append(f"{i}: {line}")

                return "\n".join(result)
            else:
                # Return all lines with line numbers
                lines = content.split("\n")
                result = []
                for i, line in enumerate(lines, 1):
                    result.append(f"{i}: {line}")

                return "\n".join(result)

        except UnicodeDecodeError:
            raise UnicodeDecodeError(
                "utf-8",
                b"",
                0,
                1,
                "File contains non-text content and cannot be displayed.",
            )
        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot access file.")
        except Exception as e:
            raise type(e)(str(e))

    def str_replace(self, file_path: str, old_str: str, new_str: str) -> str:
        """
        Replace text in a file with validation to prevent ambiguous replacements.
        
        Parameters:
        - file_path: Relative path to file to edit
        - old_str: Text to search for and replace
        - new_str: Replacement text
        
        Returns: Success message
        
        Raises: ValueError if match count is not exactly 1, FileNotFoundError, PermissionError
        
        Note: Creates automatic backup before modification. Fails if old_str appears 
        multiple times to prevent accidental replacements.
        """
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            # Read current content
            with open(abs_path, "r", encoding="utf-8") as f:
                content = f.read()

            # Count matches to ensure exactly one match
            match_count = self._count_matches(content, old_str)

            if match_count == 0:
                raise ValueError(
                    "No match found for replacement. Please check your text and try again."
                )
            elif match_count > 1:
                raise ValueError(
                    f"Found {match_count} matches for replacement text. Please provide more context to make a unique match."
                )

            # Create backup before modifying
            self._backup_file(abs_path)

            # Perform the replacement
            new_content = content.replace(old_str, new_str)

            # Write modified content back to file
            with open(abs_path, "w", encoding="utf-8") as f:
                f.write(new_content)

            return "Successfully replaced text at exactly one location."

        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot modify file.")
        except Exception as e:
            raise type(e)(str(e))

    def create(self, file_path: str, file_text: str) -> str:
        """
        Create a new file with specified content.
        
        Parameters:
        - file_path: Relative path to new file
        - file_text: Content to write to file
        
        Returns: Success message with file path
        
        Raises: FileExistsError if file already exists, PermissionError
        """
        try:
            abs_path = self._validate_path(file_path)

            # Check if file already exists
            if os.path.exists(abs_path):
                raise FileExistsError(
                    "File already exists. Use str_replace to modify it."
                )

            # Create parent directories if they don't exist
            os.makedirs(os.path.dirname(abs_path), exist_ok=True)

            # Create the file with specified content
            with open(abs_path, "w", encoding="utf-8") as f:
                f.write(file_text)

            return f"Successfully created {file_path}"

        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot create file.")
        except Exception as e:
            raise type(e)(str(e))

    def insert(self, file_path: str, insert_line: int, new_str: str) -> str:
        """
        Insert a line of text at a specified line number.
        
        Parameters:
        - file_path: Relative path to file
        - insert_line: Line number to insert after (0 = insert at beginning)
        - new_str: Text to insert
        
        Returns: Success message with line number
        
        Raises: FileNotFoundError, IndexError if line number out of range
        
        Note: Creates automatic backup before modification
        """
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            # Create backup before modifying
            self._backup_file(abs_path)

            # Read current lines from file
            with open(abs_path, "r", encoding="utf-8") as f:
                lines = f.readlines()

            # Handle line endings - add newline if needed
            if lines and not lines[-1].endswith("\n"):
                new_str = "\n" + new_str

            # Insert at the beginning if insert_line is 0
            if insert_line == 0:
                lines.insert(0, new_str + "\n")
            # Insert after the specified line number
            elif insert_line > 0 and insert_line <= len(lines):
                lines.insert(insert_line, new_str + "\n")
            else:
                raise IndexError(
                    f"Line number {insert_line} is out of range. File has {len(lines)} lines."
                )

            # Write modified lines back to file
            with open(abs_path, "w", encoding="utf-8") as f:
                f.writelines(lines)

            return f"Successfully inserted text after line {insert_line}"

        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot modify file.")
        except Exception as e:
            raise type(e)(str(e))

    def undo_edit(self, file_path: str) -> str:
        """
        Undo the last edit by restoring from backup.
        
        Parameters:
        - file_path: Relative path to file to undo changes for
        
        Returns: Success message
        
        Raises: FileNotFoundError if no backups exist
        """
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            # Restore from the most recent backup
            return self._restore_backup(abs_path)

        except ValueError as e:
            raise ValueError(str(e))
        except FileNotFoundError:
            raise FileNotFoundError("No previous edits to undo")
        except PermissionError:
            raise PermissionError("Permission denied. Cannot restore file.")
        except Exception as e:
            raise type(e)(str(e))


In [ ]:
# Tool execution dispatcher for handling text editor commands
# Processes tool use requests from Claude and executes the appropriate operations

import json

# Initialize the TextEditorTool instance for use throughout the session
text_editor_tool = TextEditorTool()


def run_tool(tool_name, tool_input):
    """
    Execute a tool command based on the tool name and input parameters.
    
    Parameters:
    - tool_name: Name of the tool (e.g., "str_replace_editor")
    - tool_input: Dictionary containing command and parameters
      - command: One of "view", "str_replace", "create", "insert", "undo_edit"
      - path: File path (required for all commands)
      - Other parameters depend on the specific command
    
    Returns: String result from the executed command
    
    Raises: Exception if tool_name is unknown or command is invalid
    
    Supported commands:
    - view: Display file contents, optionally with line range
    - str_replace: Replace text at one location in the file
    - create: Create a new file with specified content
    - insert: Insert a line at specified line number
    - undo_edit: Restore file from backup
    """
    if tool_name == "str_replace_editor":
        command = tool_input["command"]
        
        if command == "view":
            # Display file contents with optional line range selection
            return text_editor_tool.view(
                tool_input["path"], tool_input.get("view_range")
            )
        elif command == "str_replace":
            # Replace text at a specific location (must match exactly once)
            return text_editor_tool.str_replace(
                tool_input["path"], tool_input["old_str"], tool_input["new_str"]
            )
        elif command == "create":
            # Create a new file with the specified content
            return text_editor_tool.create(tool_input["path"], tool_input["file_text"])
        elif command == "insert":
            # Insert a line of text at the specified line number
            return text_editor_tool.insert(
                tool_input["path"],
                tool_input["insert_line"],
                tool_input["new_str"],
            )
        elif command == "undo_edit":
            # Restore the file from its most recent backup
            return text_editor_tool.undo_edit(tool_input["path"])
        else:
            raise Exception(f"Unknown text editor command: {command}")
    else:
        raise Exception(f"Unknown tool name: {tool_name}")


def run_tools(message):
    """
    Process all tool use blocks in a message and execute each tool.
    
    Parameters:
    - message: Anthropic Message object containing potential tool_use blocks
    
    Returns: List of tool_result content blocks with results or errors
    
    Logic:
    1. Extract all tool_use blocks from the message content
    2. For each tool_use block, attempt to execute the tool
    3. On success, create a tool_result block with the output
    4. On failure, create a tool_result block with the error message
    5. Return all tool_result blocks to be added back to conversation
    """
    # Extract all tool_use blocks from the message
    tool_requests = [block for block in message.content if block.type == "tool_use"]
    tool_result_blocks = []

    # Process each tool request
    for tool_request in tool_requests:
        try:
            # Execute the tool and get output
            tool_output = run_tool(tool_request.name, tool_request.input)
            # Create successful tool result block
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False,
            }
        except Exception as e:
            # Create error tool result block with exception message
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True,
            }

        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks


In [ ]:
# Define the text editor tool schema for Claude to use
# This tells Claude what tool is available and how to use it

def get_text_edit_schema(model):
    """
    Get the text editor tool schema for the specified model.
    
    Parameters:
    - model: The model version being used (for future compatibility if schemas change)
    
    Returns: Dictionary defining the text_editor tool with type and name
    
    Note: The schema tells Claude that the "str_replace_based_edit_tool" is available
    for file operations. The actual implementation is in the TextEditorTool class.
    """
    return {
        "type": "text_editor_20250728",                  # Tool type for text editing capability
        "name": "str_replace_based_edit_tool",           # Tool name Claude will reference in tool_use blocks
    }


In [ ]:
# Run the conversation loop with tool support
# Enables Claude to perform file operations using the text editor tool

def run_conversation(messages):
    """
    Execute a multi-turn conversation with Claude, supporting tool use.
    
    Parameters:
    - messages: List of message dictionaries representing conversation history
    
    Returns: Updated messages list after conversation completes
    
    Logic flow:
    1. Send messages to Claude with text_editor tool enabled
    2. Add Claude's response to conversation history
    3. Print any text output from Claude's response
    4. If Claude requested a tool use (stop_reason == "tool_use"):
       a. Execute the tool request
       b. Add tool results back to conversation
       c. Loop back to step 1
    5. If Claude didn't request a tool use (normal stop_reason):
       - Exit loop and return conversation history
    
    This implements a standard agentic loop where Claude can iteratively
    use tools to accomplish tasks, with results feeding back into the conversation.
    """
    while True:
        # Send message to Claude with text editor tool available
        response = chat(
            messages,
            tools=[get_text_edit_schema(model)],
        )

        # Add Claude's response to conversation history
        add_assistant_message(messages, response)
        
        # Print any text content from the response to the user
        print(text_from_message(response))

        # Check if Claude requested a tool use
        if response.stop_reason != "tool_use":
            # Claude finished without requesting tools, exit the loop
            break

        # Claude requested tool use(s), execute them and add results back
        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

    return messages


In [ ]:
# Test the text editor tool with a sample conversation
# Initialize and run the conversation loop with the text editor tool enabled

# Initialize conversation message history
messages = []

# Add a user message with a task for Claude
# The text_editor tool will be available for Claude to use
# (Add your task/prompt here for Claude to perform file operations)
add_user_message(
    messages,
    """
    [Add your file editing task here for Claude to perform]
    """,
)

# Run the conversation, allowing Claude to use the text editor tool iteratively
# Claude can now create, view, edit, and undo changes to files
run_conversation(messages)
